In [345]:
import sys
!"{sys.executable}" -m pip install torch torchvision torchaudio
print(sys.executable)

C:\Users\garvi\anaconda3\python.exe


In [346]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

In [347]:
df=pd.read_csv('hdfc.csv')
df.drop(columns=['Adj Close'],inplace=True)
df.head()

,Date,Open,High,Low,Close,Volume
0,1996-01-01,3.030,3.030,2.925,2.980,350000.0
1,1996-01-02,2.980,3.025,2.950,2.975,412000.0
2,1996-01-03,2.975,2.995,2.950,2.985,284000.0
3,1996-01-04,2.985,2.980,2.940,2.965,282000.0
4,1996-01-05,2.965,2.980,2.950,2.960,189000.0


In [348]:
cols=list(df.columns)
df=df[cols[0:2]+[cols[4]]+cols[2:4]+[cols[5]]]
df.head()

,Date,Open,Close,High,Low,Volume
0,1996-01-01,3.030,2.980,3.030,2.925,350000.0
1,1996-01-02,2.980,2.975,3.025,2.950,412000.0
2,1996-01-03,2.975,2.985,2.995,2.950,284000.0
3,1996-01-04,2.985,2.965,2.980,2.940,282000.0
4,1996-01-05,2.965,2.960,2.980,2.950,189000.0


In [349]:
#bollinger bands strategy
def calculate_ema(series,span):
    return series.ewm(span=span, adjust=False).mean()
df['EMA12']=calculate_ema(df['Close'],12)
df['EMA26']=calculate_ema(df['Close'],26)
df['MACD']=df['EMA12']-df['EMA26']
df['Signal_Line']=calculate_ema(df['MACD'],9)
def macd_signal(row):
    if row['MACD'] > row['Signal_Line']:
        return 2
    elif row['MACD'] < row['Signal_Line']:
        return 0
    else:
        return 1
df['MACD_signal']=df.apply(macd_signal,axis=1)
df['next_day_return']=(df['Close'].shift(-1)-df['Close'])/df['Close']
df.head()

,Date,Open,Close,High,Low,Volume,EMA12,EMA26,MACD,Signal_Line,MACD_signal,next_day_return
0,1996-01-01,3.030,2.980,3.030,2.925,350000.0,2.980000,2.980000,0.000000,0.000000,1,-0.001678
1,1996-01-02,2.980,2.975,3.025,2.950,412000.0,2.979231,2.979630,-0.000399,-0.000080,0,0.003361
2,1996-01-03,2.975,2.985,2.995,2.950,284000.0,2.980118,2.980027,0.000091,-0.000046,2,-0.006700
3,1996-01-04,2.985,2.965,2.980,2.940,282000.0,2.977792,2.978914,-0.001122,-0.000261,0,-0.001686
4,1996-01-05,2.965,2.960,2.980,2.950,189000.0,2.975055,2.977513,-0.002458,-0.000700,0,-0.011824


In [350]:
def calculate_bb(df,period=20):
    df['SMA']=df['Close'].rolling(window=period).mean()
    df['STD']=df['Close'].rolling(window=period).std()
    df['Upper_band']= df['SMA']+2*df['STD']
    df['Lower_band']= df['SMA']-2*df['STD']
    return df

In [351]:
def bb_strategy(row):
    close=row['Close']
    upper=row['Upper_band']
    lower=row['Lower_band']
    if pd.isna('upper') or pd.isna('lower'):
        return 0

    if close<upper:
        return 0
    elif lower>close:
        return 2
    else:
        return 1

In [352]:
df=calculate_bb(df,period=20)
df['bb_signal']=df.apply(bb_strategy,axis=1)
df.head()

,Date,Open,Close,High,Low,Volume,EMA12,EMA26,MACD,Signal_Line,MACD_signal,next_day_return,SMA,STD,Upper_band,Lower_band,bb_signal
0,1996-01-01,3.030,2.980,3.030,2.925,350000.0,2.980000,2.980000,0.000000,0.000000,1,-0.001678,NaN,NaN,NaN,NaN,1
1,1996-01-02,2.980,2.975,3.025,2.950,412000.0,2.979231,2.979630,-0.000399,-0.000080,0,0.003361,NaN,NaN,NaN,NaN,1
2,1996-01-03,2.975,2.985,2.995,2.950,284000.0,2.980118,2.980027,0.000091,-0.000046,2,-0.006700,NaN,NaN,NaN,NaN,1
3,1996-01-04,2.985,2.965,2.980,2.940,282000.0,2.977792,2.978914,-0.001122,-0.000261,0,-0.001686,NaN,NaN,NaN,NaN,1
4,1996-01-05,2.965,2.960,2.980,2.950,189000.0,2.975055,2.977513,-0.002458,-0.000700,0,-0.011824,NaN,NaN,NaN,NaN,1


In [353]:
def calculate_rsi(df,period=14):
    delta=df['Close'].diff()
    gain= delta.where(delta>0,0.0)
    loss= -delta.where(delta<0,0.0)
    avg_gain = gain.rolling(window=period).mean()
    avg_loss = loss.rolling(window=period).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    df['RSI'] = rsi
    return df

In [354]:
def rsi_strategy(row):
    if row['RSI']>70:
        return 0
    elif row['RSI']<30:
        return 2
    else:
        return 1

In [355]:
df=calculate_rsi(df,period=14)
df['rsi_signal']=df.apply(rsi_strategy,axis=1)
df.head()

,Date,Open,Close,High,Low,Volume,EMA12,EMA26,MACD,Signal_Line,MACD_signal,next_day_return,SMA,STD,Upper_band,Lower_band,bb_signal,RSI,rsi_signal
0,1996-01-01,3.030,2.980,3.030,2.925,350000.0,2.980000,2.980000,0.000000,0.000000,1,-0.001678,NaN,NaN,NaN,NaN,1,NaN,1
1,1996-01-02,2.980,2.975,3.025,2.950,412000.0,2.979231,2.979630,-0.000399,-0.000080,0,0.003361,NaN,NaN,NaN,NaN,1,NaN,1
2,1996-01-03,2.975,2.985,2.995,2.950,284000.0,2.980118,2.980027,0.000091,-0.000046,2,-0.006700,NaN,NaN,NaN,NaN,1,NaN,1
3,1996-01-04,2.985,2.965,2.980,2.940,282000.0,2.977792,2.978914,-0.001122,-0.000261,0,-0.001686,NaN,NaN,NaN,NaN,1,NaN,1
4,1996-01-05,2.965,2.960,2.980,2.950,189000.0,2.975055,2.977513,-0.002458,-0.000700,0,-0.011824,NaN,NaN,NaN,NaN,1,NaN,1


In [356]:
def final_decision(row):
    signals=[row['MACD_signal'],row['bb_signal'],row['rsi_signal']]
    buy=signals.count(2)
    sell=signals.count(0)
    if buy>sell:
        return 2
    elif sell>buy:
        return 0
    else :
        return 1
df['final_signal'] = df.apply(final_decision, axis=1)
df.head()

,Date,Open,Close,High,Low,Volume,EMA12,EMA26,MACD,Signal_Line,MACD_signal,next_day_return,SMA,STD,Upper_band,Lower_band,bb_signal,RSI,rsi_signal,final_signal
0,1996-01-01,3.030,2.980,3.030,2.925,350000.0,2.980000,2.980000,0.000000,0.000000,1,-0.001678,NaN,NaN,NaN,NaN,1,NaN,1,1
1,1996-01-02,2.980,2.975,3.025,2.950,412000.0,2.979231,2.979630,-0.000399,-0.000080,0,0.003361,NaN,NaN,NaN,NaN,1,NaN,1,0
2,1996-01-03,2.975,2.985,2.995,2.950,284000.0,2.980118,2.980027,0.000091,-0.000046,2,-0.006700,NaN,NaN,NaN,NaN,1,NaN,1,2
3,1996-01-04,2.985,2.965,2.980,2.940,282000.0,2.977792,2.978914,-0.001122,-0.000261,0,-0.001686,NaN,NaN,NaN,NaN,1,NaN,1,0
4,1996-01-05,2.965,2.960,2.980,2.950,189000.0,2.975055,2.977513,-0.002458,-0.000700,0,-0.011824,NaN,NaN,NaN,NaN,1,NaN,1,0


In [357]:
def evaluate_trade(row, future_window=5, threshold=0.02):
    try:
        idx = df.index.get_loc(row.name)
        current_price = row['Close']
        future_prices = df['Close'].iloc[idx+1 : idx+1+future_window]

        if len(future_prices) == 1:
            return 1  # not enough future data

        future_return = (future_prices.max() - current_price) / current_price
        future_drop = (future_prices.min() - current_price) / current_price

        if row['final_signal'] == 2 and future_return > threshold:
            return 2
        elif row['final_signal'] == 0 and future_drop < -threshold:
            return 0
        else:
            return 1
    except:
        return 1



df['target'] = df.apply(evaluate_trade, axis=1)
df.head()

,Date,Open,Close,High,Low,Volume,EMA12,EMA26,MACD,Signal_Line,...,next_day_return,SMA,STD,Upper_band,Lower_band,bb_signal,RSI,rsi_signal,final_signal,target
0,1996-01-01,3.030,2.980,3.030,2.925,350000.0,2.980000,2.980000,0.000000,0.000000,...,-0.001678,NaN,NaN,NaN,NaN,1,NaN,1,1,1
1,1996-01-02,2.980,2.975,3.025,2.950,412000.0,2.979231,2.979630,-0.000399,-0.000080,...,0.003361,NaN,NaN,NaN,NaN,1,NaN,1,0,0
2,1996-01-03,2.975,2.985,2.995,2.950,284000.0,2.980118,2.980027,0.000091,-0.000046,...,-0.006700,NaN,NaN,NaN,NaN,1,NaN,1,2,1
3,1996-01-04,2.985,2.965,2.980,2.940,282000.0,2.977792,2.978914,-0.001122,-0.000261,...,-0.001686,NaN,NaN,NaN,NaN,1,NaN,1,0,0
4,1996-01-05,2.965,2.960,2.980,2.950,189000.0,2.975055,2.977513,-0.002458,-0.000700,...,-0.011824,NaN,NaN,NaN,NaN,1,NaN,1,0,0


In [358]:
df['final_signal'].value_counts()[0]


4057

In [359]:
df['final_signal'].value_counts()[1]

2315

In [360]:
df['target'].value_counts()
df.head()

,Date,Open,Close,High,Low,Volume,EMA12,EMA26,MACD,Signal_Line,...,next_day_return,SMA,STD,Upper_band,Lower_band,bb_signal,RSI,rsi_signal,final_signal,target
0,1996-01-01,3.030,2.980,3.030,2.925,350000.0,2.980000,2.980000,0.000000,0.000000,...,-0.001678,NaN,NaN,NaN,NaN,1,NaN,1,1,1
1,1996-01-02,2.980,2.975,3.025,2.950,412000.0,2.979231,2.979630,-0.000399,-0.000080,...,0.003361,NaN,NaN,NaN,NaN,1,NaN,1,0,0
2,1996-01-03,2.975,2.985,2.995,2.950,284000.0,2.980118,2.980027,0.000091,-0.000046,...,-0.006700,NaN,NaN,NaN,NaN,1,NaN,1,2,1
3,1996-01-04,2.985,2.965,2.980,2.940,282000.0,2.977792,2.978914,-0.001122,-0.000261,...,-0.001686,NaN,NaN,NaN,NaN,1,NaN,1,0,0
4,1996-01-05,2.965,2.960,2.980,2.950,189000.0,2.975055,2.977513,-0.002458,-0.000700,...,-0.011824,NaN,NaN,NaN,NaN,1,NaN,1,0,0


In [361]:
df['final_signal'].value_counts()


final_signal
0    4057
1    2315
2     195
Name: count, dtype: int64

In [362]:
df.shape

(6567, 21)

In [363]:
df = df.dropna()


In [364]:
x_train,x_test,y_train,y_test=train_test_split(df.iloc[:,1:20],df.iloc[:,20], test_size=0.66)

In [365]:
scaler=StandardScaler()
x_train=scaler.fit_transform(x_train)
x_test=scaler.transform(x_test)

In [366]:
x_train

array([[-0.74215755, -0.74124513, -0.74224213, ..., -1.76920643,
         2.22323532, -0.73723531],
       [ 1.32679374,  1.31507032,  1.30811142, ..., -2.16597187,
         2.22323532, -0.73723531],
       [ 0.2969967 ,  0.28524496,  0.28981586, ..., -2.26689256,
         2.22323532, -0.73723531],
       ...,
       [ 0.5468043 ,  0.55166316,  0.55516974, ..., -1.3344171 ,
         0.2041368 , -0.73723531],
       [-0.62805701, -0.62820031, -0.62896076, ..., -0.59545663,
         0.2041368 , -0.73723531],
       [-0.1873068 , -0.18207878, -0.18579725, ...,  0.82775434,
         0.2041368 ,  1.13031305]])

In [367]:
y_train

1279    0
5599    1
5107    1
6273    0
1926    1
928     1
1898    1
3161    1
139     0
4113    1
1540    1
4057    1
6249    0
653     1
2573    1
4996    1
6000    1
6171    1
1262    1
1913    1
755     1
2902    1
1405    1
4774    1
4629    1
1257    1
2548    0
5381    1
1627    0
4597    0
5797    1
4537    1
4887    1
5765    1
5853    1
1941    1
4759    1
150     0
3828    1
2388    1
2370    1
1464    1
369     1
5453    1
5106    0
6128    1
1030    0
1061    1
1057    1
3829    1
409     1
5944    0
5317    1
3728    1
850     1
3871    1
1957    1
1895    1
4215    1
5068    1
5815    1
633     0
1376    1
5330    1
2213    1
795     1
1563    1
1874    1
5709    0
484     0
1498    0
5299    1
2141    0
6193    0
3135    0
1584    1
3203    1
3913    0
2771    1
5312    1
1285    1
4432    0
2774    1
1927    0
35      0
703     1
5130    1
1277    0
3371    1
957     1
4673    0
4275    1
123     1
5182    1
1659    1
678     1
6456    1
4022    0
3525    1
5555    1


In [368]:
print("Unique values in df['target'] before splitting:", df['target'].unique())


Unique values in df['target'] before splitting: [1 0 2]


In [369]:
x_train_tensor=torch.from_numpy(x_train).float()
x_test_tensor=torch.from_numpy(x_test).float()
y_train_tensor = torch.from_numpy(y_train.to_numpy()).long()
y_test_tensor = torch.from_numpy(y_test.to_numpy()).double()


In [370]:
x_train_tensor

tensor([[-0.7422, -0.7412, -0.7422,  ..., -1.7692,  2.2232, -0.7372],
        [ 1.3268,  1.3151,  1.3081,  ..., -2.1660,  2.2232, -0.7372],
        [ 0.2970,  0.2852,  0.2898,  ..., -2.2669,  2.2232, -0.7372],
        ...,
        [ 0.5468,  0.5517,  0.5552,  ..., -1.3344,  0.2041, -0.7372],
        [-0.6281, -0.6282, -0.6290,  ..., -0.5955,  0.2041, -0.7372],
        [-0.1873, -0.1821, -0.1858,  ...,  0.8278,  0.2041,  1.1303]])

In [373]:
import torch
import torch.nn as nn
import torch.optim as optim
class MySimpleNN(nn.Module):
    def __init__(self, input_dim, output_dim=3):
        super(MySimpleNN, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)
        
    def forward(self, x):
        return self.linear(x)  # raw logits, no softmax

model = MySimpleNN(x_train_tensor.shape[1])
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

epochs = 30
for epoch in range(epochs):
    outputs = model(x_train_tensor)
    loss = criterion(outputs, y_train_tensor)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")


Epoch 1, Loss: 1.2121
Epoch 2, Loss: 1.1876
Epoch 3, Loss: 1.1673
Epoch 4, Loss: 1.1487
Epoch 5, Loss: 1.1295
Epoch 6, Loss: 1.1100
Epoch 7, Loss: 1.0917
Epoch 8, Loss: 1.0748
Epoch 9, Loss: 1.0588
Epoch 10, Loss: 1.0432
Epoch 11, Loss: 1.0277
Epoch 12, Loss: 1.0127
Epoch 13, Loss: 0.9984
Epoch 14, Loss: 0.9848
Epoch 15, Loss: 0.9719
Epoch 16, Loss: 0.9595
Epoch 17, Loss: 0.9472
Epoch 18, Loss: 0.9352
Epoch 19, Loss: 0.9236
Epoch 20, Loss: 0.9124
Epoch 21, Loss: 0.9017
Epoch 22, Loss: 0.8914
Epoch 23, Loss: 0.8813
Epoch 24, Loss: 0.8714
Epoch 25, Loss: 0.8618
Epoch 26, Loss: 0.8525
Epoch 27, Loss: 0.8436
Epoch 28, Loss: 0.8349
Epoch 29, Loss: 0.8264
Epoch 30, Loss: 0.8181


In [374]:
print("Unique values in y_train:", y_train_tensor.unique())


Unique values in y_train: tensor([0, 1, 2])


In [375]:
model.eval()  # evaluation mode
with torch.no_grad():
    y_test_pred = model(x_test_tensor)
    y_test_pred_labels = torch.argmax(y_test_pred, dim=1)
from sklearn.metrics import accuracy_score

# Convert tensors to numpy arrays if needed
y_true = y_test_tensor.numpy()
y_pred = y_test_pred_labels.numpy()

accuracy = accuracy_score(y_true, y_pred)
print(f"Test Accuracy: {accuracy * 100:.2f}%")


Test Accuracy: 74.15%
